In [1]:
import os
import re
import fitz
print(os.getcwd())

C:\Users\jessi\OneDrive\School\lab\Data Extraction from PDF\code\pdf-extraction\backend\module


In [2]:
pdfFile = '../data/demo/2c7d75bb5c074aff8a5e1a24c628bf39/test/公務員懲戒委員會102年度清字第11347號公懲裁定.pdf'
pdf_text = ''
if os.path.exists(pdfFile):
    print(pdfFile)
    with fitz.open(pdfFile) as pdf:
        for page in pdf:
            pdf_text = pdf_text + page.get_text("text")
else:
    print('File is not exist!')

new_pdf_text = re.sub(r'^(\s\s|)[0-9][0-9]+\n', '', pdf_text)
new_pdf_text = re.sub(r'[１-９]+\n', '', new_pdf_text)
new_pdf_text = re.sub(r'[0-9]+\n', '', new_pdf_text)
new_pdf_text = re.sub(r' ', '', new_pdf_text)
new_pdf_text = re.sub(r'^　', '', new_pdf_text)
new_pdf_text = re.sub(r'^\n', '', new_pdf_text)
# print(new_pdf_text)
pdf_text = new_pdf_text
len(pdf_text)
# print(pdf_text)

../data/demo/2c7d75bb5c074aff8a5e1a24c628bf39/test/公務員懲戒委員會102年度清字第11347號公懲裁定.pdf


627

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1500,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(pdf_text)
len(texts)

1

In [4]:
# len(texts[0])
texts[0]

'公務員懲戒委員會裁定\n\u3000\n102年度清字第11347號\n移送機關\u3000內政部\u3000\u3000設臺北市○○區○○路○號\n代\u3000表\u3000人\u3000徐國勇\u3000\u3000住同上\n被付懲戒人\u3000陳仁維\u3000\u3000臺東縣警察局警務員\n上列被付懲戒人因違法失職案件，經內政部移送審理，本會裁定\n如下：\n主文\n本件停止審議程序之議決撤銷。\n理由\n一、按公務員懲戒法中華民國104年5月1日修正之條文施行前已\n繫屬於公務員懲戒委員會之懲戒案件尚未終結者，於該法修\n正施行後，由公務員懲戒委員會合議庭依修正後之程序規定\n繼續審理。但修正施行前已依法進行之程序，其效力不受影\n響。為該法第77條第1款所明定。又停止審理程序之裁定，\n公務員懲戒委員會合議庭得依聲請或依職權撤銷之。同法第\n39條第2項定有明文。\n二、本件被付懲戒人陳仁維因違法失職案件應否受懲戒處分及處\n分之輕重，同一行為涉及刑事部分，以其犯罪是否成立為斷\n，前經本會於102年3月15日議決於刑事裁判確定前，停止審\n議程序在案。經查被付懲戒人所涉刑事案件，業經臺灣高等\n法院花蓮分院以106年度重上更�字第17號判決在案，此有\n該判決書在卷可稽。依上開說明，本會合議庭自得依職權將\n原停止審議程序之議決撤銷，繼續審理程序。\n三、依公務員懲戒法第39條第2項，裁定如主文。\n中華民國108年9月11日\n公務員懲戒委員會第二庭\n審判長委員姜仁脩\n委員吳景源\n委員洪佳濱\n委員吳謀焰\n委員蘇振堂\n以上正本證明與原本無異。\n中華民國108年9月12日\n書記官黃紋麗'

In [ ]:
# len(texts[1])
texts[1]

In [5]:
with open("API_KEY", "r", encoding="utf-8") as f:
    key = f.read()

In [6]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=key)
docsearch = FAISS.from_texts(texts, embeddings)

In [7]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.prompts import PromptTemplate

prompt_template = """使用PDF文件的上下文來回答問題，並完全擷取文件的文字內容且不重複問題，而不知道答案就說不知道，或者在文件中找不到答案，請不要試圖編答案

{context}

Question: {question}
Answer in traditional Chinese:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [8]:
chain = load_qa_chain(ChatOpenAI(model_name="gpt-3.5-turbo",openai_api_key=key), chain_type="stuff", prompt=PROMPT)

In [9]:
def query_func(query):
  with get_openai_callback() as cb:
    docs = docsearch.similarity_search(query)
    res = chain.run(input_documents=docs, question=query)
    print(f"Output: {res}")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [10]:
query_func("請告訴我文件中的title")

Output: 公務員懲戒委員會裁定
Total Tokens: 1014
Prompt Tokens: 995
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.002028


In [11]:
query_func("請告訴我文件中的標題")

Output: 公務員懲戒委員會裁定
Total Tokens: 1018
Prompt Tokens: 999
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.002036


In [12]:
query_func("請告訴我文件中的title(標題)")

Output: 公務員懲戒委員會裁定
Total Tokens: 1020
Prompt Tokens: 1001
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.0020399999999999997


In [13]:
query_func("title指文件中的標題，請告訴我文件中的title")

Output: 公務員懲戒委員會裁定
Total Tokens: 1025
Prompt Tokens: 1006
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.0020499999999999997


In [14]:
query_func("請告訴我文件中的main_content")

Output: 本件停止審議程序之議決撤銷，繼續審理程序。被付懲戒人陳仁維因違法失職案件應否受懲戒處分及處分之輕重，同一行為涉及刑事部分，以其犯罪是否成立為斷，前經本會於102年3月15日議決於刑事裁判確定前，停止審議程序在案。本會合議庭自得依職權將原停止審議程序之議決撤銷，繼續審理程序。依公務員懲戒法第39條第2項，裁定如主文。
Total Tokens: 1226
Prompt Tokens: 996
Completion Tokens: 230
Successful Requests: 1
Total Cost (USD): $0.0024519999999999998


In [15]:
query_func("請告訴我文件中的主文")

Output: 本件停止審議程序之議決撤銷。
Total Tokens: 1016
Prompt Tokens: 996
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.002032


In [16]:
query_func("請告訴我文件中的main_content(主文)")

Output: 本件停止審議程序之議決撤銷。
Total Tokens: 1019
Prompt Tokens: 999
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.0020380000000000003


In [17]:
query_func("main_content指文件中的主文，請告訴我文件中的main_content")

Output: 本件停止審議程序之議決撤銷。
Total Tokens: 1025
Prompt Tokens: 1005
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.0020499999999999997


In [18]:
query_func("請告訴我文件中的date")

Output: 中華民國108年9月11日
Total Tokens: 1007
Prompt Tokens: 995
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.002014


In [19]:
query_func("請告訴我文件中的日期")

Output: 中華民國108年9月11日
Total Tokens: 1007
Prompt Tokens: 995
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.002014


In [20]:
query_func("請告訴我文件中的date(日期)")

Output: 中華民國108年9月11日。
Total Tokens: 1010
Prompt Tokens: 997
Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.00202


In [21]:
query_func("date指文件中的日期，請告訴我文件中的date")

Output: 中華民國108年9月11日。
Total Tokens: 1015
Prompt Tokens: 1002
Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.00203


In [22]:
query_func("請告訴我文件中的appeal.appellant")

Output: 文件中沒有提到appeal或appellant，因此無法回答此問題。
Total Tokens: 1023
Prompt Tokens: 998
Completion Tokens: 25
Successful Requests: 1
Total Cost (USD): $0.002046


In [23]:
query_func("請告訴我文件中的移送機關/上訴人/聲請人/原告")

Output: 移送機關：內政部
上訴人：無
聲請人：無
原告：無
Total Tokens: 1049
Prompt Tokens: 1016
Completion Tokens: 33
Successful Requests: 1
Total Cost (USD): $0.002098


In [24]:
query_func("請告訴我文件中的移送機關")

Output: 內政部
Total Tokens: 1005
Prompt Tokens: 1001
Completion Tokens: 4
Successful Requests: 1
Total Cost (USD): $0.00201


In [25]:
query_func("請告訴我文件中的appeal.appellant(移送機關/上訴人/聲請人/原告)")

Output: 文件中沒有提到appeal、appellant、移送機關、上訴人、聲請人或原告。
Total Tokens: 1057
Prompt Tokens: 1021
Completion Tokens: 36
Successful Requests: 1
Total Cost (USD): $0.002114


In [26]:
query_func("請告訴我文件中的appeal.appellant(移送機關)")

Output: 移送機關為內政部。
Total Tokens: 1021
Prompt Tokens: 1006
Completion Tokens: 15
Successful Requests: 1
Total Cost (USD): $0.002042


In [27]:
query_func("appeal.appellant指文件中的移送機關/上訴人/聲請人/原告，請告訴我文件中的appeal.appellant")

Output: 文件中的appeal.appellant為代表人徐國勇。
Total Tokens: 1048
Prompt Tokens: 1028
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.002096


In [28]:
query_func("請告訴我文件中的代表人")

Output: 徐國勇。
Total Tokens: 1004
Prompt Tokens: 997
Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $0.0020080000000000002


In [29]:
query_func("請告訴我文件中的被付懲戒人")

Output: 被付懲戒人為陳仁維，為臺東縣警察局警務員。
Total Tokens: 1045
Prompt Tokens: 1003
Completion Tokens: 42
Successful Requests: 1
Total Cost (USD): $0.00209


In [30]:
query_func("請告訴我文件中的date")

Output: 中華民國108年9月11日。
Total Tokens: 1008
Prompt Tokens: 995
Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.002016


In [31]:
query_func("請告訴我文件中的理由項目有多少個?並請一一列出")

Output: 文件中的理由項目共有三個，分別為：
1. 依據公務員懲戒法的相關條文，已進行程序的效力不受影響，且停止審議程序的議決可以撤銷。
2. 涉及刑事部分的案件需要等待刑事裁判確定後再進行審議，並已有相關判決書在案，因此可以撤銷原本的停止審議程序。
3. 根據公務員懲戒法第39條第2項的規定，裁定如主文。
Total Tokens: 1211
Prompt Tokens: 1014
Completion Tokens: 197
Successful Requests: 1
Total Cost (USD): $0.0024219999999999997


In [32]:
query_func("請一一列出文件中理由的項目")

Output: 一、公務員懲戒法修正前已繫屬於公務員懲戒委員會之懲戒案件尚未終結者，於該法修正施行後，由公務員懲戒委員會合議庭依修正後之程序規定繼續審理。但修正施行前已依法進行之程序，其效力不受影響。
二、本件被付懲戒人陳仁維因違法失職案件應否受懲戒處分及處分之輕重，同一行為涉及刑事部分，以其犯罪是否成立為斷，前經本會於102年3月15日議決於刑事裁判確定前，停止審議程序在案。
三、被付懲戒人所涉刑事案件已有判決確定，本會合議庭可依職權將原停止審議程序之議決撤銷，繼續審理程序。
Total Tokens: 1339
Prompt Tokens: 999
Completion Tokens: 340
Successful Requests: 1
Total Cost (USD): $0.0026780000000000003


In [33]:
query_func("請一一列出文件中理由的項目，並不做其他修改")

Output: 1. 公務員懲戒法修正施行前已繫屬於公務員懲戒委員會之懲戒案件尚未終結者，於該法修正施行後，由公務員懲戒委員會合議庭依修正後之程序規定繼續審理。
2. 停止審理程序之裁定，公務員懲戒委員會合議庭得依聲請或依職權撤銷之。
3. 本件被付懲戒人陳仁維因違法失職案件應否受懲戒處分及處分之輕重，同一行為涉及刑事部分，以其犯罪是否成立為斷，前經本會於102年3月15日議決於刑事裁判確定前，停止審議程序在案。
Total Tokens: 1307
Prompt Tokens: 1007
Completion Tokens: 300
Successful Requests: 1
Total Cost (USD): $0.0026139999999999996


In [35]:
query_func("請問文件中的理由")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).
Retrying

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60)

In [ ]:
query_func("請問文件中的審判長委員")

In [ ]:
query_func("請問文件中的法庭")

In [ ]:
query_func("請問文件中的委員有哪些")

In [ ]:
query_func("請問文件中的委員")

In [ ]:
query_func("請問文件中的書記官")

In [ ]:
query_func("請告訴我文件中的書記官")

In [36]:
query_func("請擷取理由正則式的表示方法")


Output: 按公務員懲戒法中華民國104年5月1日修正之條文施行前已繫屬於公務員懲戒委員會之懲戒案件尚未終結者，於該法修正施行後，由公務員懲戒委員會合議庭依修正後之程序規定繼續審理。但修正施行前已依法進行之程序，其效力不受影響。為該法第77條第1款所明定。又停止審理程序之裁定，公務員懲戒委員會合議庭得依聲請或依職權撤銷之。同法第39條第2項定有明文。
Total Tokens: 1247
Prompt Tokens: 1000
Completion Tokens: 247
Successful Requests: 1
Total Cost (USD): $0.002494
